# ОПИСАНИЕ ПРОЕКТА

**ЗАДАЧА**: Построить скоринг модель для вторичных клиентов банка, которая бы предсказывала вероятность дефолта клиента. Для этого нужно будет определить значимые параметры заемщика.

**ЭТАПЫ РАБОТЫ**:

1. Первичная визуализация - выводы о качестве данных, распределение целевой переменной;
2. Построение наивной модели + метрики;
3. EDA: визуализация, выбросы, пропуски, корреляционный анализ, генерация признаков; 
4. Оценка важности;
5. Построение модели логистической регрессии;
6. Подбор гиперпараметров;
7. Эксперименты с другими моделями, определение лучшей;
8. Финальная модель, submission. 

**ОПИСАНИЯ ПОЛЕЙ**:

* **client_id** - идентификатор клиента
* **education** - уровень образования
* **sex** - пол заемщика
* **age** - возраст заемщика
* **car** - флаг наличия автомобиля
* **car_type** - флаг автомобиля иномарки
* **decline_app_cnt** - количество отказанных прошлых заявок
* **good_work** - флаг наличия “хорошей” работы
* **bki_request_cnt** - количество запросов клиента в БКИ о своей кредитной истории
* **home_address** - категоризатор домашнего адреса
* **work_address** - категоризатор рабочего адреса
* **income** - доход заемщика
* **foreign_passport** - наличие загранпаспорта
* **sna** - связь заемщика с клиентами банка
* **first_time** - давность наличия информации о заемщике
* **score_bki** - скоринговый балл по данным из БКИ
* **region_rating** - рейтинг региона
* **app_date** - дата подачи заявки
* **default** - флаг дефолта по кредиту

# Используемые библиотеки

In [ ]:
import numpy as np
import pandas as pd
import pandas_profiling as pp
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import auc, roc_auc_score, roc_curve, confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import f1_score, mean_squared_error

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.linear_model import LogisticRegression

# Вспомогательные функции

In [ ]:
def visualise_metrics(model, X, y):
    '''Функция для разделения выборки на train и valid'''
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, random_state=RANDOM_SEED, test_size=0.2)

    # Обучение модели на тренировочных данных и предсказание на валидационных
    model.fit(X_train, y_train)
    probs = model.predict_proba(X_valid)[:, 1]
    y_pred = model.predict(X_valid)

    # Вывод типа модели:
    print()
    print('Model Type: ' + str(model))
    print()

    # Вывод confusion matrix:
    sns.set_context(context='paper', font_scale=2, rc=None)
    group_names = ['True\nPositive', 'False\nPositive',
                   'False\nNegative', 'True\nNegative']
    group_counts = ['{0:0.0f}'.format(
        value) for value in confusion_matrix(y_valid, y_pred).flatten()]
    labels = [f'{v1}\n\n{v2}' for v1, v2 in zip(group_names, group_counts)]
    labels = np.asarray(labels).reshape(2, 2)
    ax = sns.heatmap(confusion_matrix(y_valid, y_pred),
                     annot=labels, fmt='', cmap='Reds')
    ax.set(xlabel='predicted', ylabel='real', title='Confusion Matrix')
    plt.show()
    print()

    # Вывод значений метрик:
    print('accuracy_score:\t\t {:.3}'.format(accuracy_score(y_valid, y_pred)))
    print('f1_score:\t\t {:.3}'.format(f1_score(y_valid, y_pred)))
    print('precision_score:\t {:.3}'.format(precision_score(y_valid, y_pred)))
    print('recall_score:\t\t {:.3}'.format(recall_score(y_valid, y_pred)))
    print('roc_auc_score:\t\t {:.3}'.format(roc_auc_score(y_valid, probs)))
    print()
    

def show_outliers(data, column): 
    '''Функция для рассчета выбросов методом IQR'''
    q25, q75 = data[column].quantile([0.25, 0.75])
    IQR = q75 - q25
    low_limit = q25 - 1.5 * IQR
    up_limit = q75 + 1.5 * IQR
    print("IQR range [{}, {}]".format(low_limit, up_limit),
                       "\nMin. value: {} \nMax. value: {}".
    format(data[column].min(), data[column].max()))
    print("Number below the lower limit: {}, number above the upper limit: {}".
    format(data[data[column] < low_limit][column].count(),
           data[data[column] > up_limit][column].count()))

# Оценка датасета

In [ ]:
RANDOM_SEED=42

In [ ]:
data = pd.read_csv("/kaggle/input/sf-dst-scoring/train.csv")
test_data = pd.read_csv("/kaggle/input/sf-dst-scoring/test.csv")
sample = pd.read_csv('/kaggle/input/sf-dst-scoring/sample_submission.csv')

In [ ]:
# Для корректной обработки объединяем данные
data['sample'] = 1  # Помечаем тренировочные данные
test_data['sample'] = 0  # Помечаем тестовые данные
test_data['default'] = -1

df = test_data.append(data, sort=False).reset_index(drop=True)

In [ ]:
print(data.info())
print('data size: ', data.shape)
data.head()

In [ ]:
print(test_data.info())
print('test_data size: ', test_data.shape)
test_data.head(5)

In [ ]:
print(sample.info())
print(sample.shape)
sample.head(5)

В датасете всего **18 переменных** (помимо переменной **sample**, разделяющей тренировочную и тестовую выборку) и целевая переменная **default**.

# 1. Первичная визуализация

In [ ]:
pp.ProfileReport(df)

Имеются признаки с различными типами данных - **числовые**, **категориальные** и **бинарные**.

Иcходя из контекста и описания, переменные **car** и **car_type** дублируют информацию друг друга. Возможно, эти две переменные будут иметь значительную коллинеарность (проверим позже).

**Бинарные** признаки будут обработаны с помощью **LabelEncoder'a**, а из **категориальных** признаков будут созданы **dummy переменные**.

In [ ]:
fig, ax = plt.subplots(figsize=(20, 12))
sns_heatmap = sns.heatmap(
    df.isnull(), yticklabels=False, cbar=False, cmap='viridis')

Есть пропущенные значения в столбце **education**. Их количество несущественно.

Посмотрим на распределение признака **default**:

In [ ]:
data.default.hist()

Недефолтных клиентов у банка значительно больше, чем дефолтных. Данные несбалансированны.

Сгруппируем признаки в три категории по типу их обработки (**категориальные**, **бинарные** и **числовые**):

In [ ]:
# Бинарные переменные
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

# Категориальные переменные
cat_cols = ['education', 'work_address', 'home_address', 'sna', 'app_date']

# Числовые переменные
num_cols = ['age', 'decline_app_cnt', 'bki_request_cnt',
            'income', 'score_bki', 'region_rating', 'first_time']

Смотрим распределение **числовых признаков**:

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(26,17))
for i, col in enumerate(num_cols):
    sns.histplot(df[col], ax=axes.flat[i]).set(title='Histplot for ' + col)

In [ ]:
# Распределение дефолтных клиентов относительно числовых признаков
fig, axes = plt.subplots(2, 3, figsize=(25,18))
for i, col in enumerate(num_cols[:-1]):
    sns.boxplot(y = data[col], x = 'default', data=data, ax=axes.flat[i],
                      palette='rainbow').set(title='Boxplot for ' + col)

* Видим, что распределения всех чиловых признаков, кроме **score_bki**, имеют тяжелый правый хвост.
* Дефолтные клиенты в среднем младше, имеют большее количество отклоненных заявок и больше запросов в БКИ. А также в среднем имеют более низкий доход.
* В **region_rating** хоть данные и предоставлены в числовом виде, имеется некоторая тенденция к категориальности, где значения распределены от 20 до 80 с шагом в 10.

Оценим корреляцию числовых признаков:

In [ ]:
sns.set(font_scale=1)
plt.subplots(figsize=(12, 12))
sns.heatmap(df[num_cols].corr().abs(), vmin=0, vmax=1, square=True,
                             annot=True, fmt=".1f", linewidths=0.1,
    cmap="RdBu").set_title('График корреляций числовых переменных')

In [ ]:
imp_num = pd.Series(f_classif(df[num_cols], df['default'])[0], index=num_cols)
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')

Так как **категориальные** признаки требуют обработки, рассмотрим их подробнее позже.

# Построение наивной модели

Займемся преобразованием **бинарных признаков**:

In [ ]:
label_encoder = LabelEncoder()
for column in bin_cols:
    df[column] = label_encoder.fit_transform(df[column])

Удалим **education** из-за наличия пропусков. Также удалим **app_date** и **client_id**:

In [ ]:
df_naiv = df.drop(['client_id', 'education', 'app_date'], axis=1)

In [ ]:
train_processed = df_naiv.query('sample == 1').drop(['sample'], axis=1)
test_processed = df_naiv.query('sample == 0').drop(['sample'], axis=1)

In [ ]:
# Удалим для X целевую переменную
X = train_processed.drop(columns=['default'])
y = train_processed['default']

In [ ]:
lr = LogisticRegression(random_state=RANDOM_SEED, max_iter=1000)
visualise_metrics(lr, X, y)

Наивная модель выдает кредит абсолютно всем и предсказывает отсутствие дефолтных клиентов. Идеально, но несопоставимо с реальностью.

# EDA: визуализация, выбросы, пропуски, корреляционный анализ, генерация признаков

# Выбросы

In [ ]:
# Смотрим на выбросы числовых признаков
for col in num_cols:
    print(col)
    show_outliers(df, col)
    print("\n")

В переменной **decline_app_cnt** 83% значений нулевые. Убирать все значения выше 0 - абсолютно лишено смысла. 

Посмотрим на процентное распределение уникальных значений:

In [ ]:
df['decline_app_cnt'].value_counts(normalize=True)

Значения выше 4 встречаются реже, чем у 0.5% данных. Заменяем все значения выше 4 на 4 и использовать как категориальный признак при построении модели.

In [ ]:
df.loc[df['decline_app_cnt'] > 4, 'decline_app_cnt'] = 4

In [ ]:
# Посмотрим на результат
df['decline_app_cnt'].value_counts(normalize=True)

Похожая ситуация с переменной **bki_request_cnt**. Проделаем то же самое.

In [ ]:
df['bki_request_cnt'].value_counts(normalize=True)

In [ ]:
# Определим порог значением 5 
df.loc[df['bki_request_cnt'] > 5, 'bki_request_cnt'] = 5

In [ ]:
df['income'].value_counts(normalize=True)

В случае **income**, порог в 90000 так же не особенно реалистичен. Исходя из графиков распределения, условная граница, после которой почти нет значений - 0.4. Выберем верхнюю границу в 500к и заменим этим значением все, что выше.

In [ ]:
# Определим порог значением 500000
df.loc[df['income'] > 500000, 'income'] = 500000

Установим нижнюю границу **score_bki** согласно IQR range. Для верхней границы используем значение 0.

In [ ]:
df.loc[df['score_bki'] < -3.299, 'score_bki'] = -3.299
df.loc[df['score_bki'] > 0, 'score_bki'] = 0

* Признак **education**

In [ ]:
df.education.value_counts().plot.barh()

Заполним пропуски самым распространенным значением - "**SCH**":

In [ ]:
df.education = df.education.fillna(df.education.mode()[0])
df.isnull().sum()

Закодируем  признак **education**:

In [ ]:
label_encoder = LabelEncoder()
df['education'] = label_encoder.fit_transform(df['education'])
df.head()

* Распределения всех чиловых признаков, кроме **score_bki**, имеют тяжелый правый хвост. Прологарифмируем их позже, чтобы уйти от этой проблемы. **score_bki** исключим из логарифмирования, так как он может принимать отрицательные значения.

In [ ]:
for i in num_cols:
    if i != 'score_bki':
        df[i] = np.log(1 + df[i])

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(26,17))
for i, col in enumerate(num_cols):
    sns.histplot(df[col], ax=axes.flat[i]).set(title='Histplot for ' + col)

Сработало только с признаком **income**.

In [ ]:
# Распределение дефолтных клиентов относительно числовых признаков
fig, axes = plt.subplots(2, 3, figsize=(25,18))
for i, col in enumerate(num_cols[:-1]):
    sns.boxplot(y = data[col], x = 'default', data=data, ax=axes.flat[i],
                      palette='rainbow').set(title='Boxplot for ' + col)

На боксплотах видно большое количество выбросов в **score_bki**, **income**, **bki_request_cnt** и **decline_app_cnt**.

* Признак **app_date**

In [ ]:
# Преобразуем в формат даты
df['app_date'] = pd.to_datetime(df['app_date'])

# Вычислим количество дней с самой давней записи в датасете 
df['app_days'] = df['app_date'].apply(lambda x: x - df['app_date'].min())
df['app_days'] = df['app_days'].dt.days
df.drop('app_date', axis=1, inplace=True)

# Обновим список с числовыми признаками
cat_cols = list(set(cat_cols) - set(['app_date']))
num_cols = num_cols + ['app_days']

# Оценка важности

Оценим корреляцию и значимость **числовых признаков**:

In [ ]:
sns.set(font_scale=1)
plt.subplots(figsize=(12, 12))
sns.heatmap(df[num_cols].corr().abs(), vmin=0, vmax=1, square=True,
                             annot=True, fmt=".1f", linewidths=0.1,
    cmap="RdBu").set_title('График корреляций числовых переменных')

In [ ]:
imp_num = pd.Series(f_classif(df[num_cols], df['default'])[0], index=num_cols)
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')

Как мы видим, взаимосвязь пар числовых признаков по Пирсону слабая, что прекрасно для нашей модели! 
**score_bki** - наиболее значимый числовой признак, **age** - наименее значимый.

Оценим корреляцию и значимость **категориальных и бинарных признаков**:

In [ ]:
sns.set(font_scale=1)
plt.subplots(figsize=(12, 12))
sns.heatmap(df[cat_cols + bin_cols].corr().abs(), vmin=0, vmax=1, square=True,
                             annot=True, fmt=".1f", linewidths=0.1,
     cmap="RdBu").set_title('Корреляция категориальных и бинарных переменных')

Наблюдается высокая корреляция между признаками **home_addres/work_address**, **car/car_type**. Посмотрим на значимость признаков и решим, какие отбросить, а какие оставить.

In [ ]:
imp_cat = pd.Series(mutual_info_classif(df[bin_cols + cat_cols], df['default'],
                           discrete_features=True), index=bin_cols + cat_cols)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh')

Согласно значимости оставляем - **home_address** и **car_type**, удаляем - **work_address** и **car**. Также удаляем столбец **client_id**.

In [ ]:
df = df.drop(df[['work_address', 'car_type', 'client_id']], axis=1)
cat_cols = list(set(cat_cols) - set(['work_address']))
bin_cols = list(set(bin_cols) - set(['car_type']))

Перед обучением регрессии стандартизируем **числовые признаки**, а для **категориальных переменных** используем dummy-кодирование:

In [ ]:
X_num = StandardScaler().fit_transform(df[num_cols].values)
df[num_cols] = X_num

In [ ]:
df = pd.get_dummies(df, columns=cat_cols)
df.head()

# Построение модели логистической регрессии

In [ ]:
# Обновляем списки переменных
num_cols = ['age', 'score_bki', 'region_rating', 'first_time', 'app_days']
cat_cols = ['sna', 'home_address', 'education',
            'decline_app_cnt', 'bki_request_cnt', 'income']
bin_cols = ['foreign_passport', 'sex', 'good_work', 'car']

In [ ]:
train_processed = df.query('sample == 1').drop(['sample'], axis=1)
test_processed = df.query('sample == 0').drop(['sample'], axis=1)

# Удалим для X целевую переменную 
X = train_processed.drop(columns=['default'])
y = train_processed['default']

lr = LogisticRegression(random_state=RANDOM_SEED, max_iter=1000)
visualise_metrics(lr, X, y)

Предсказания модели улучшились, но всё еще неприменимы в реальной жизни. Улучшаем!

# Эксперименты

In [ ]:
lr_balanced = LogisticRegression(
    class_weight='balanced', max_iter=1000, random_state=RANDOM_SEED)
visualise_metrics(lr_balanced, X, y)

В сбалансированной модели резко уменьшается количество ошибок первого рода, но появляется значительное количество ошибок второго рода.

# Подбор гиперпараметров

In [ ]:
# Создадим набор гиперпараметров
hyperparameters = {'C': np.logspace(-4, 4, 20)}

X = train_processed.drop(columns=['default'])
y = train_processed['default']

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, random_state=RANDOM_SEED, test_size=0.2)

model = LogisticRegression(random_state=RANDOM_SEED,
                           class_weight='balanced', max_iter=1000)

# Создадим сетку поиска с использованием 5-кратной перекрёстной проверки
clf = GridSearchCV(model, hyperparameters, cv=5, verbose=0, scoring='roc_auc')

best_model = clf.fit(X_train, y_train)

# Вывод лучшего параметра С
print('Лучшее C:', best_model.best_estimator_.get_params()['C'])

In [ ]:
lr_best = LogisticRegression(random_state=RANDOM_SEED,
    class_weight='balanced', max_iter=1000, C=0.012742749857031334)
visualise_metrics(lr_best, X, y)

Попробуем применить **undersampling**:

Оставим случайным образом в тренировочных данных столько строк с нулевым дефолтом, сколько всего есть данных меньшего класса

In [ ]:
train_unders = pd.concat(
 [train_processed[train_processed['default'] == 0].sample(len(
 train_processed[train_processed['default'] == 1]), random_state=RANDOM_SEED),
                            train_processed[train_processed['default'] == 1]])

X = train_unders.drop(columns=['default'])
y = train_unders['default']

lr_balanced = LogisticRegression(
    class_weight='balanced', max_iter=1000, random_state=RANDOM_SEED)
visualise_metrics(lr_balanced, X, y)

Применение **undersampling** позволило значительно улучшить предсказания модели. Теперь верные предсказания по дефолтным и недефолтным клиентам превосходят ошибочные. Но показатель ROC AUC стал меньше.

# Выводы

* После обработки данных (**EDA**) построили базовую модель логистической регрессии, которая показала **ROC AUC** > 0.74, а также **accuracy** 0.874, **precision** 0.363, **recall** 0.020 и **f1_score** 0.0384
* При подборе гиперпараметра с использованием **GridSearchCV** и **логарифмсеткой** (-4,4) получили, что при значении **C** = 0.01274 модель показывает практически тот же результат. **ROC AUC** не изменился, **f1** примерно такой же 0.338.
* Также, для сравнения, построили модель логистической регрессии с одинаковым количеством классов целевой переменной. **f1** стал еще лучше 0,67 за счет изменения **recall_score**, но **ROC AUC** стал хуже 0,733. Поэтому в итоге взяли базовую модель с гиперпараметром  **C** = 0.01274.

# Финальная модель и submission

In [ ]:
X_test = test_processed.drop(columns=['default'])
y_pred = lr_best.predict_proba(X_test)
results_df = pd.DataFrame(
    data={'client_id': test_data['client_id'], 'default': y_pred[:, 1]})
results_df.to_csv('submission.csv', index=False)
results_df